In [ ]:
"""
Train the model using Yolov8 and Roboflow

Reference: https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset/
"""

In [1]:
# if not installed run: pip install ultralytics
import ultralytics
print(ultralytics.checks())

Ultralytics YOLOv8.0.232 🚀 Python-3.10.12 torch-2.2.0+cu121 CPU (AMD Ryzen 9 6900HS with Radeon Graphics)
Setup complete ✅ (16 CPUs, 14.9 GB RAM, 77.9/97.5 GB disk)
None


In [5]:
# if not installed run: pip install roboflow
# if you have a weird 'appengine' error run this: pip install urllib3==2.0.7
# other errors, run through chat gpt
# this was copied from exporting the dataset on roboflow with 'yolov8'

from roboflow import Roboflow

rf = Roboflow(api_key="tG58USi1UKOd6l1rkJEw") # Private Key for Team
project = rf.workspace("arcturus-gx59n").project("buoy-labeling-full")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.232, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Buoy-Labeling-Full-1 in yolov8:: 100%|██████████| 2006/2006 [00:00<00:00, 11454.81it/s]


In [ ]:
# Run the training of the model with the correct location to the data.yaml and custom settings (could take a while)
# This assumes data folder is in perception folder and this document is in perception folder
data_folder = 'Buoy-Labeling-Full-1'

imgsz = 640

model= 'yolov8n.pt' # the other models are too big

# ! symbol lets you run command line interface commands (terminal commands) 
# Run this in command line if you get errors
# replace data.location with the name of the file folder
# WARNING: This will take a while and consume a lot of computer resources
!yolo task=detect mode=train model=yolov8n.pt data={data.location}/data.yaml epochs=25 imgsz=640 plots=True
